All topological realated functions are used from the package giotto-tda

Reference: giotto-tda: A Topological Data Analysis Toolkit for Machine Learning and Data Exploration, Tauzin et al, arXiv:2004.02551, 2020.

Some part of the code were take from the tutorials and examples of giotto-tda. Reference: https://giotto-ai.github.io/gtda-docs/latest/notebooks/index.html

In [9]:
!Run this if giott-tda is not installed
!pip install giotto-tda
!pip install keras
!pip3 install opencv-python
!pip install librosa
from pyentrp import entropy as ent
!pip install pyentrp

'Run' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23706 sha256=0e83d862bc1d0af94a66a58caf7c700fe9bff40a6cdb8037fbda18bc3389d21a
  Stored in directory: c:\users\lucky\appdata\local\pip\cache\wheels\e4\76\a4\cfb55573167a1f5bde7d7a348e95e509c64b2c3e8f921932c3
Successfully built audioread


ModuleNotFoundError: No module named 'pyentrp'

In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from   scipy import signal
# from keras.models import Sequential
# from keras.layers import Dense,LSTM,Dropout
# from keras.layers.embeddings import Embedding
# from keras.preprocessing import  sequence

In [12]:
print('Resample to Frequency Rate ... ')
start_time    = time.time()                # To keep track of time to run the code
def re_sample(name):
    data1         = pd.read_csv(name) # Loading Accelerometer data
    acc_x         = data1.acc_x
    time_series   = data1.ts_receiver
    sample_rate   = len(time_series)/(time_series[len(time_series)-1] - time_series[0])
    signal_len    = len(acc_x)
    resample_rate = 256
    resample_len  = int(signal_len / sample_rate * 256)
    return resample_len

def sliding_window_seg(data_s, windowSize, windowstep):  # 200Hz data
    data = np.array(data_s)
    w_range = int((len(data) - windowSize + windowstep)/windowstep)
    window_var = []
    a = 0
    b = a + windowSize - 1
    i = 0
    while i < w_range:
        w_data = data[a:b]
        window_var.append(np.var(w_data))
        a = a + windowstep
        b = b + windowstep
        i = i + 1
    window_var = np.array(window_var)
    threshold = 420  # set larger threshold for a more fined segmentation
    var_b = window_var < threshold
    var_b = var_b * 1

    # check whether there are fake pauses, which has pause time < 3 windows (~300ms)
    pauses = []
    pauses_count = 0
    for i in range(len(var_b)):
        if var_b[i] == 1:
            pauses.append(i)
        else:
            if 0 < len(pauses) <= 3:
                var_b[pauses] = 0
                pauses_count += 1
            pauses = []
    print('Detected and removed {} fake pauses'.format(pauses_count))
    segment = {}  # segment data where number of (var_b == 0) > 500ms
    seg_point = []
    count = 0
    for i in range(len(var_b)):
        # print(var_b[i])
        if var_b[i] == 0:
            seg_point.append(i)
        else:
            if seg_point:
                count += 1
                segment[count] = seg_point
                seg_point = []
    if var_b[-1] == 0:
        count += 1
        segment[count] = seg_point
    start_end = []
    for i in range(len(segment)):
        seg = np.array(segment[i+1])
        if len(seg) > 5:  #around 300ms
            start = seg[0] * windowstep
            end = seg[-1] * windowstep + windowSize - 1
            start_end.append([start, end])
    return start_end, threshold

Resample to Frequency Rate ... 


In [13]:
import numpy as np
from astropy.stats import median_absolute_deviation
from scipy.stats import pearsonr
from scipy.stats import skew
from scipy.stats import kurtosis
from librosa.feature import mfcc
# from pyentrp import entropy as ent

# def minind_acc(x):
#     x_feat = [i.index.get_loc(i.idxmin()) for i in x]
#     return x_feat

# def maxind_acc(x):
#     x_feat = [i.index.get_loc(i.idxmax()) for i in x]
#     return x_feat

# def sma_acc(x, y, z):
#     sma = [sum(abs(x[i])+abs(y[i])+abs(z[i])) for i in range(len(x))]
#     return sma

# def svm_acc(x, y, z):
#     svm = [sum(x[i]**2 + y[i]**2 + z[i]**2) for i in range(len(x))]
#     return svm

# def cor_acc(x, y, z):
#     xy_feat = [pearsonr(x[i],y[i])[0] for i in range(len(x))]
#     yz_feat = [pearsonr(y[i],z[i])[0] for i in range(len(y))]
#     xz_feat = [pearsonr(x[i],z[i])[0] for i in range(len(z))]
#     return xy_feat, yz_feat, xz_feat

# def mag_acc(x, y, z):
#     mpre = x*x+y*y+z*z
#     m = np.sqrt(mpre)
#     m_smooth = m
# #    m_smooth[2:(len(m_smooth)-2)] = [(m[i-2] + m[i-1] +m[i] + m[i+1] + m[i+2])/5 for i ,val in enumerate(m[2:(len(m)-2)])]
#     return m_smooth

# def mfcc_acc(x):
#     x_feat = mfcc(x.values)
#     return x_feat

epsn = 1e-12 # to avoid dividing zero.

def mean_acc(x):
    x_feat = np.mean(x)
    return x_feat

def max_acc(x):
    x_feat = np.max(x)
    return x_feat

def min_acc(x):
    x_feat = np.min(x)
    return x_feat

def median_acc(x):
    x_feat = np.median(x)
    return x_feat

def std_acc(x):
    x_feat = np.std(x)
    return x_feat

def energy_acc(x):
    x_feat = np.sum(np.power(x,2))
    return x_feat

def med_abs_dev_acc(x):
    x_feat = median_absolute_deviation(x)
    return x_feat

def perc_acc(x, percent):
    x_feat = np.percentile(x,percent)
    return x_feat

def iqr_acc(x):
    x_feat = np.percentile(x,75)-np.percentile(x,25)
    return x_feat

def ptop_acc(x):
    x_feat = np.max(x)-np.min(x)
    return x_feat

def cross_zero_acc(x):
    count = 0;
    for i in range(len(x)-1):
        if x[i] * x[i+1] < 0:
            count = count + 1 
    return count

def kurt_acc(x):
    x_feat = kurtosis(x)
    return x_feat

def kurt_index(x):
    n = len(x)
    temp1 = np.sum((x-mean_acc(x))**4)
    temp2 = (np.sqrt(var_acc(x)))**4
    return temp1/((n-1)*temp2)

def skew_acc(x):
    x_feat = skew(x)
    return x_feat

def skew_index(x):
    n     = len(x)
    temp1 = np.sum(( x - mean_acc(x) )**3)
    temp2 = (np.sqrt(var_acc(x)))**3
    return temp1/((n-1)*temp2)

def entropy_acc(x):
    x_feat = ent.permutation_entropy(x)
    return x_feat

def mean_abs_diff(x):
    x_feat = np.mean(np.absolute(x - np.mean(x)))
    return x_feat

def max_min_diff(x):
    x_feat = max(x) - min(x)
    return x_feat

def med_abs_dev(x):
    x_feat = np.median(np.absolute(x - np.median(x)))
    return x_feat

def neg_count(x):
    count = 0;
    for i in range(len(x)):
        if x[i] < 0:
            count = count + 1
    return count
    
def pos_count(x):
    count = 0;
    for i in range(len(x)):
        if x[i] > 0:
            count = count + 1
    return count

def cross_mean(x):
    count = 0;
    for i in range(len(x)-1):
        if (x[i] - np.mean(x)) * (x[i+1] - np.mean(x)) < 0:
            count = count + 1 
    return count

def above_mean(x):
    count = 0;
    for i in range(len(x)):
        if (x[i] - np.mean(x)) > 0:
            count = count + 1 
    return count

def below_mean(x):
    count = 0;
    for i in range(len(x)):
        if (x[i] - np.mean(x)) < 0:
            count = count + 1 
    return count

def number_peaks(x):
    return len(find_peaks(x))

def rms_acc(x):
    return np.sqrt(np.mean(np.square(x)))

def sr_acc(x): # square root
    return np.square(np.mean(np.sqrt(np.abs(x))))

def am_acc(x):
    return np.mean(np.abs(x))

def waveform_index(x):
    return rms_acc(x)/(am_acc(x)+epsn)

def peak_index(x):
    return max_acc(x)/(rms_acc(x)+epsn)

def impluse_factor(x):
    return max_acc(x)/(am_acc(x)+epsn)

def tolerance_index(x):
    return max_acc(x)/(sr_acc(x)+epsn)

# ###****### features for FFT domain
def fft_fft(x):
    fft_trans = np.abs(np.fft.fft(x))
    freq_spectrum = fft_trans[1:int(np.floor(len(x) * 1.0 / 2)) + 1]
    _freq_sum_ = np.sum(freq_spectrum)
    return freq_spectrum, _freq_sum_            
              
def fft_mean(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    return np.mean(freq_spectrum)
              
def fft_var(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    return np.var(freq_spectrum)
              
def fft_abs_diff(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    return np.mean(np.absolute(freq_spectrum - np.mean(freq_spectrum)))

def fft_min(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    return np.min(freq_spectrum)

def fft_min(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    return np.max(freq_spectrum)

def fft_max_min_diff(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    return np.max(freq_spectrum) - np.min(freq_spectrum)

def fft_median(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    return np.median(freq_spectrum)

def fft_med_abs_dev(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    x_feat = median_absolute_deviation(freq_spectrum)
    return x_feat

def fft_iqr(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    x_feat = np.percentile(freq_spectrum,75)-np.percentile(freq_spectrum,25)
    return x_feat

def fft_cross_mean(x):
    count = 0;
    freq_spectrum, _freq_sum_ = fft_fft(x)
    for i in range(len(freq_spectrum)-1):
        if (freq_spectrum[i] - np.mean(freq_spectrum)) * (freq_spectrum[i+1] - np.mean(freq_spectrum)) < 0:
            count = count + 1 
    return count

def fft_above_mean(x):
    count = 0
    freq_spectrum, _freq_sum_ = fft_fft(x)
    for i in range(len(freq_spectrum)):
        if (freq_spectrum[i] - np.mean(freq_spectrum)) > 0:
            count = count + 1 
    return count

def fft_below_mean(x):
    count = 0
    freq_spectrum, _freq_sum_ = fft_fft(x)
    for i in range(len(freq_spectrum)):
        if (freq_spectrum[i] - np.mean(freq_spectrum)) < 0:
            count = count + 1 
    return count

def fft_number_peaks(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    return len(find_peaks(freq_spectrum))

def fft_rms(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    return np.sqrt(np.mean(np.square(freq_spectrum)))

def fft_sr(x): # square root
    freq_spectrum, _freq_sum_ = fft_fft(x)
    return np.square(np.mean(np.sqrt(np.abs(freq_spectrum))))

def fft_am(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    return np.mean(np.abs(freq_spectrum))

def fft_kurt(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    x_feat = kurtosis(freq_spectrum)
    return x_feat

def fft_kurt_index(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    n = len(freq_spectrum)
    temp1 = np.sum(( freq_spectrum - mean_acc( freq_spectrum ))**4)
    temp2 = (np.sqrt(var_acc( freq_spectrum )))**4
    return temp1 / ((n-1)*temp2)

def fft_skew(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    x_feat = skew(freq_spectrum)
    return x_feat

def fft_skew_index(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    n     = len(freq_spectrum)
    temp1 = np.sum((freq_spectrum - mean_acc(freq_spectrum))**3)
    temp2 = (np.sqrt( var_acc(freq_spectrum) ))**3
    return temp1/((n-1)*temp2)

def fft_shape_mean(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    shape_sum = np.sum([xi * freq_spectrum[xi] for xi in range(len(freq_spectrum))])
    return 0 if _freq_sum_ < epsn else shape_sum * 1.0 / _freq_sum_

def fft_shape_std(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    shape_mean = fft_shape_mean(sequence_data)
    var = np.sum([0 if _freq_sum_ < epsn else np.power((x - shape_mean), 2) * freq_spectrum[x] for x in range(len(freq_spectrum))]) / _freq_sum_
    return np.sqrt(var)

def fft_shape_skew(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    shape_mean = fft_shape_mean(sequence_data)
    return np.sum([np.power((x - shape_mean), 3) * freq_spectrum[x] for x in range(len(freq_spectrum))]) / _freq_sum_

def fft_shape_kurt(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    shape_mean = fft_shape_mean(sequence_data)
    return np.sum([np.power((x - shape_mean), 4) * freq_spectrum[x] - 3 for x in range(len(freq_spectrum))]) / _freq_sum_

def fft_energy(x):
    freq_spectrum, _freq_sum_ = fft_fft(x)
    return np.sum(freq_spectrum ** 2) / len(freq_spectrum)

def fft_entropy(sequence_data):
    freq_spectrum, _freq_sum_ = fft_fft(sequence_data)
    pr_freq = freq_spectrum * 1.0 / _freq_sum_
    entropy = -1 * np.sum([np.log2(p+1e-5) * p for p in pr_freq])
    return entropy

# ###****### features for Time-Frequency domain




In [48]:
from scipy.signal import wiener
from scipy.ndimage import gaussian_filter


def plot_stft_sig(data2, name1):
    resample_rate = 256
    sp,th = sliding_window_seg(data2[1:,1], 340, 40)  #(acc_x)
    for seg_channel in range(len(sp)):
        for acc_channel in range(3):
            jpg_file = name1 + str(seg_channel) + '_' + str(acc_channel) + '.jpg'
            acc_x = data2[:,acc_channel]
            zz    = acc_x[sp[0][0]:sp[0][1]]
            print(zz.shape,resample_rate)
            PER_SEG    = 128
            if len(zz) > PER_SEG:
                Fs         = resample_rate
                f, t, Zxx  = signal.stft( zz, Fs, nperseg=PER_SEG,noverlap = int(0.99*PER_SEG) )
                print('the shape of f is', f.shape)
                f          = f[0:12]
                Zxx        = Zxx[0:12,:]
                Zxx        = wiener(Zxx,[8,8])
                Zxx        = gaussian_filter(Zxx, sigma=2)
                fig = plt.figure(figsize=(8,4))
                ax  = plt.Axes(fig, [0., 0., 1., 1.])
                ax.set_axis_off()
                fig.add_axes(ax)
                plt.pcolormesh(t, f, np.abs(Zxx), vmin=0, vmax=64, shading='gouraud')
                plt.savefig(jpg_file, transparent=True, bbox_inches='tight', pad_inches=0)
                plt.close("all")
                # plt.show()

In [49]:
filePath = 'D:/test/data_set/subject2/word/sub'
names    = os.listdir(filePath);

In [50]:
path = "D:/test/picture"
os.chdir(path)
os.getcwd()

'D:\\test\\picture'

In [51]:
import os
print(names)
check = ['hand']
for nn in range(len(names)):
    curr_file = filePath + '/' + names[nn];
    (filePathin1, tempfilename1) = os.path.split(curr_file)
    print('curren_file_is', curr_file)
    print('the current temp file name is', tempfilename1)

    session = tempfilename1.split('_')[-2]
    print('the session is', session)
    if session in check:
        name      = curr_file
        left      = tempfilename1.split('_')[:-2]
        left_right = ''
        for left_i in range(len(left)):
            left_right = left_right + left[left_i] + '_' 
        name_hand  = curr_file
        name_left  = filePath + '/' + left_right + 'leftear_s1.csv'
        name_right = filePath + '/' + left_right + 'rightear_s1.csv'
        print('the name of hand should be',name_hand)
        resample_len  = re_sample(name_hand)
        print('the resample length should be', resample_len)
        data2         = pd.read_csv(name_hand)
        data2         = data2.drop(['ts_receiver','ts_sensor','gyr_x','gyr_y','gyr_z','mag_x','mag_y','mag_z'], axis=1)
        data2         = signal.resample(data2, resample_len, axis = 0)
        ###***### get the accelerometer and geroscope of facial expressions
        ## following part is for left ear part
        print('the name left is i', name_left)
        print('the left right is', left_right)
        data3         = pd.read_csv(name_left)
        data3         = data3.drop(['ts_receiver','ts_sensor','mag_x','mag_y','mag_z'], axis=1)
        data3         = signal.resample(data3, resample_len, axis = 0)
        ## following part is for right ear part
        data4         = pd.read_csv(name_right)
        data4         = data4.drop(['ts_receiver','ts_sensor','mag_x','mag_y','mag_z'], axis=1)
        data4         = signal.resample(data4, resample_len, axis = 0)
        # name_to_save  = 'jpg' + '/' + left_right
        name_to_save  = left_right
        plot_stft_sig(data2, name_to_save)
        # print(sp,th)

['.ipynb_checkpoints', 'jpg_file', 's2_CHAPLATE_hand_s1.csv', 's2_CHAPLATE_leftear_s1.csv', 's2_CHAPLATE_rightear_s1.csv', 's2_CSCARRY_hand_s1.csv', 's2_CSCARRY_leftear_s1.csv', 's2_CSCARRY_rightear_s1.csv', 's2_MMDRIVE_hand_s1.csv', 's2_MMDRIVE_leftear_s1.csv', 's2_MMDRIVE_rightear_s1.csv', 's2_NOTTYPE_hand_s1.csv', 's2_NOTTYPE_leftear_s1.csv', 's2_NOTTYPE_rightear_s1.csv', 's2_QSHE_hand_s1.csv', 's2_QSHE_leftear_s1.csv', 's2_QSHE_rightear_s1.csv', 's2_THCARRY_hand_s1.csv', 's2_THCARRY_leftear_s1.csv', 's2_THCARRY_rightear_s1.csv']
curren_file_is D:/test/data_set/subject2/word/sub/.ipynb_checkpoints
the current temp file name is .ipynb_checkpoints
the session is .ipynb
curren_file_is D:/test/data_set/subject2/word/sub/jpg_file
the current temp file name is jpg_file
the session is jpg
curren_file_is D:/test/data_set/subject2/word/sub/s2_CHAPLATE_hand_s1.csv
the current temp file name is s2_CHAPLATE_hand_s1.csv
the session is hand
the name of hand should be D:/test/data_set/subject2/wor

In [23]:
# a New Method is 
# def deframesig(frames, siglen, frame_len, frame_step, winfunc=lambda x: numpy.ones((x,))):
#     """Does overlap-add procedure to undo the action of framesig.
#     :param frames: the array of frames.
#     :param siglen: the length of the desired signal, use 0 if unknown. Output will be truncated to siglen samples.
#     :param frame_len: length of each frame measured in samples.
#     :param frame_step: number of samples after the start of the previous frame that the next frame should begin.
#     :param winfunc: the analysis window to apply to each frame. By default no window is applied.
#     :returns: a 1-D signal.
#     """
#     frame_len = round_half_up(frame_len)
#     frame_step = round_half_up(frame_step)
#     numframes = numpy.shape(frames)[0]
#     assert numpy.shape(frames)[1] == frame_len, '"frames" matrix is wrong size, 2nd dim is not equal to frame_len'

#     indices = numpy.tile(numpy.arange(0, frame_len), (numframes, 1)) + numpy.tile(
#         numpy.arange(0, numframes * frame_step, frame_step), (frame_len, 1)).T
#     indices = numpy.array(indices, dtype=numpy.int32)
#     padlen = (numframes - 1) * frame_step + frame_len

#     if siglen <= 0: siglen = padlen

#     rec_signal = numpy.zeros((padlen,))
#     window_correction = numpy.zeros((padlen,))
#     win = winfunc(frame_len)

#     for i in range(0, numframes):
#         window_correction[indices[i, :]] = window_correction[
#                                                indices[i, :]] + win + 1e-15  # add a little bit so it is never zero
#         rec_signal[indices[i, :]] = rec_signal[indices[i, :]] + frames[i, :]

#     rec_signal = rec_signal / window_correction
#     return rec_signal[0:siglen]

In [349]:
## detected every activities based on the hand gesture activity detection
# data       = data2[1:,0]
# seg_points = sp
# axis_x     = None
# axis_x = np.arange(0, len(data))
# for i in range(len(seg_points)):
#     print(i)
#     if axis_x is None:
#         axis_x = np.arange(0, len(data))
#     fig, ax = plt.subplots(figsize=(10, 5))
#     ax.plot(axis_x, data)
#     ax.vlines(axis_x[seg_points[i][0]], min(data), max(data), 'r', 'dashed', label='Start')
#     ax.vlines(axis_x[seg_points[i][1]], min(data), max(data), 'r', 'dashed', label='end')
#     plt.legend()
#     plt.show()